In [16]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 4.1 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=08c020aad9ca38c2803a1417b0cd9b2ccb1c20650e9a578a32e846a7369bab32
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

ln: failed to create symbolic link '/mydrive/MyDrive': File exists
'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_UK
 a0529-a0611_US		       Github
 ccc_compiled.csv	       heatmap529_UK.ipynb
 csv			       heatmap530.ipynb
 csv.zip		       Heat_Map_Online_language.ipynb
 dataframe_csv		       __MACOSX
 dataframe_csv_EU	       mapping_UK
 dataframe_csv_UK	       mapping_US
 dataframe_result_529_UK.csv   non_violant1.csv
 dataframe_result_530_UK.csv   non_violant2.csv
 dataframe_result_531_UK.csv   UK
 dataframe_result_csv	       violant.csv
 dataframe_result_csv_EU      '新建 Microsoft PowerPoint 演示文稿.pptx'


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_601 = 'csv/US05_29.csv'

In [8]:
ct = pd.read_csv(ct_path_601)

In [9]:
ct.shape[1]

47

In [10]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,104380826317363_2958316794257071,Facebook,2020-05-29 22:10:01,2021-06-15 16:21:42,link,Aftershocks of George Floyd's death hit Centra...,theledger.com,George Floyd attended South Florida State Coll...,George Floyd attended South Florida State Coll...,1.0,https://www.theledger.com/news/20200529/afters...,https://www.facebook.com/104380826317363/posts...,3375,-1.250000,1.0,5,7,0,0,2,0,2,0,0,0,3,2,2,2,2,3,2,4,0,1,23719,News Chief,newschief,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The News Chief is a daily newspaper in Winter ...,False,2011-04-27 15:29:39,en,0,23719|2958316794257071
1,2015249512117888_2353472281628941,Facebook,2020-05-29 22:10:00,2021-07-14 01:56:00,link,Justice — Zina for Congress,zinaforcongress.com,Protesters taking to the streets to seek justi...,"Ahmaud Arbery, Breonna Taylor and George Floyd...",2.0,http://www.zinaforcongress.com/justice,https://www.facebook.com/2015249512117888/post...,796,1.826087,1.0,31,6,0,5,0,0,0,0,0,1,6,2,2,3,2,3,2,3,0,2,8581055,Zina Spezakis for Congress NJ9,zspezakisNJ,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,Our leadership is ineffective and corrupt. It ...,False,2019-02-12 20:21:37,en,0,8581055|2353472281628941
2,127784800585461_3342576742439568,Facebook,2020-05-29 22:10:00,2021-07-20 14:56:16,link,A Fearless Response To George Floyd’s Death | ...,fearlessfollower.org,A Fearless Response To George Floyd’s Death by...,Hey Fearless Followers! Here's a challenging r...,1.0,https://fearlessfollower.org/a-fearless-respon...,https://www.facebook.com/127784800585461/posts...,4484,3.263158,1.0,16,29,4,11,0,0,2,0,0,0,3,3,2,3,0,0,0,0,0,1,1467090,Ridge Point Community Church,ridgepointchurch,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Ridge Point exists to Follow Jesus and Fearles...,False,2010-06-14 17:44:07,en,0,1467090|3342576742439568
3,7127779172_10158347173219173,Facebook,2020-05-29 22:09:59,2021-07-19 06:12:54,link,The Shooting Of Black Americans Started Long B...,pridenation.com,By Taryn Finley | HuffPost Donald Trump threat...,The Shooting Of Black Americans Started Long B...,2.0,http://ow.ly/8IiY50zUbyT,https://www.facebook.com/7127779172/posts/1015...,2459,-0.900000,1.0,0,0,0,0,0,0,1,0,0,0,1,2,1,1,1,1,2,1,0,1,8609875,PNN Media Group,pridenation,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,PHARMATIVE PNN Media Group is the parent compa...,False,2008-01-07 06:37:40,en,0,8609875|10158347173219173
4,57218502143_10157744314187144,Facebook,2020-05-29 22:09:54,2021-08-06 00:24:46,link,CN2 Newscast 5/29/20 - CN2 News,cn2.com,A pastor spreading a message of unity and love...,A pastor spreading a message of unity and love...,2.0,https://www.cn2.com/cn2-newscast-5-29-20/,https://www.facebook.com/57218502143/posts/101...,24960,-6.000000,1.0,7,0,0,0,0,0,0,0,0,0,8,6,3,4,4,4,4,9,0,3,1486517,CN2 News,CN2RockHill,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,NEWS TIP? 803-326-2762 or news@cn2.com\n\n,False,2009-03-09 22:18:59,en,0,1486517|10157744314187144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23895,203828796317254_3382699755096793,Faceb

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,"{'Avon Park': 1, 'Polk County': 1, 'Minneapoli...","{'Avon Park': 1, 'Polk County': 1}"
1,{'America': 1},{}
2,{},{}
3,{},{}
4,{},{}


In [24]:
ct.to_csv('a0529.csv')

In [25]:
ct=pd.read_csv('a0529.csv')

In [26]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_UK
 a0529-a0611_US		       Github
 a0529.csv		       heatmap529_UK.ipynb
 ccc_compiled.csv	       heatmap530.ipynb
 csv			       Heat_Map_Online_language.ipynb
 csv.zip		       __MACOSX
 dataframe_csv		       mapping_UK
 dataframe_csv_EU	       mapping_US
 dataframe_csv_UK	       non_violant1.csv
 dataframe_result_529_UK.csv   non_violant2.csv
 dataframe_result_530_UK.csv   UK
 dataframe_result_531_UK.csv   violant.csv
 dataframe_result_csv	      '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_result_csv_EU


In [27]:
###
#ct=pd.read_csv('a0601.csv')

In [28]:
ct

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,0,104380826317363_2958316794257071,Facebook,2020-05-29 22:10:01,2021-06-15 16:21:42,link,Aftershocks of George Floyd's death hit Centra...,theledger.com,George Floyd attended South Florida State Coll...,George Floyd attended South Florida State Coll...,1.0,https://www.theledger.com/news/20200529/afters...,https://www.facebook.com/104380826317363/posts...,3375,-1.250000,1.0,5,7,0,0,2,0,2,0,0,0,3,2,2,2,2,3,2,4,0,1,23719,News Chief,newschief,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The News Chief is a daily newspaper in Winter ...,False,2011-04-27 15:29:39,en,0,23719|2958316794257071,"{'Avon Park': 1, 'Polk County': 1, 'Minneapoli...","{'Avon Park': 1, 'Polk County': 1}",{}
1,1,2015249512117888_2353472281628941,Facebook,2020-05-29 22:10:00,2021-07-14 01:56:00,link,Justice — Zina for Congress,zinaforcongress.com,Protesters taking to the streets to seek justi...,"Ahmaud Arbery, Breonna Taylor and George Floyd...",2.0,http://www.zinaforcongress.com/justice,https://www.facebook.com/2015249512117888/post...,796,1.826087,1.0,31,6,0,5,0,0,0,0,0,1,6,2,2,3,2,3,2,3,0,2,8581055,Zina Spezakis for Congress NJ9,zspezakisNJ,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,Our leadership is ineffective and corrupt. It ...,False,2019-02-12 20:21:37,en,0,8581055|2353472281628941,{'America': 1},{},{}
2,2,127784800585461_3342576742439568,Facebook,2020-05-29 22:10:00,2021-07-20 14:56:16,link,A Fearless Response To George Floyd’s Death | ...,fearlessfollower.org,A Fearless Response To George Floyd’s Death by...,Hey Fearless Followers! Here's a challenging r...,1.0,https://fearlessfollower.org/a-fearless-respon...,https://www.facebook.com/127784800585461/posts...,4484,3.263158,1.0,16,29,4,11,0,0,2,0,0,0,3,3,2,3,0,0,0,0,0,1,1467090,Ridge Point Community Church,ridgepointchurch,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Ridge Point exists to Follow Jesus and Fearles...,False,2010-06-14 17:44:07,en,0,1467090|3342576742439568,{},{},{}
3,3,7127779172_10158347173219173,Facebook,2020-05-29 22:09:59,2021-07-19 06:12:54,link,The Shooting Of Black Americans Started Long B...,pridenation.com,By Taryn Finley | HuffPost Donald Trump threat...,The Shooting Of Black Americans Started Long B...,2.0,http://ow.ly/8IiY50zUbyT,https://www.facebook.com/7127779172/posts/1015...,2459,-0.900000,1.0,0,0,0,0,0,0,1,0,0,0,1,2,1,1,1,1,2,1,0,1,8609875,PNN Media Group,pridenation,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,PHARMATIVE PNN Media Group is the parent compa...,False,2008-01-07 06:37:40,en,0,8609875|10158347173219173,{},{},{}
4,4,57218502143_10157744314187144,Facebook,2020-05-29 22:09:54,2021-08-06 00:24:46,link,CN2 Newscast 5/29/20 - CN2 News,cn2.com,A pastor spreading a message of unity and love...,A pastor spreading a message of unity and love...,2.0,https://www.cn2.com/cn2-newscast-5-29-20/,https://www.facebook.com/57218502143/posts/101...,24960,-6.000000,1.0,7,0,0,0,0,0,0,0,0,0,8,6,3,4,4,4,4,9,0,3,1486517,CN2 News,CN2RockHill,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,NEWS TIP? 803-326-2762 or news@cn2.com\n\n,False,2009-03-09 22:18:59,en,0,1486517|10157744314187144,{},{},{}
...,...,...,...,...,...,...,...,...,...,...,...,

In [29]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [30]:
#ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [31]:
###
ct_ner = ct

In [32]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,"{'Avon Park': 1, 'Polk County': 1, 'Minneapoli...","{'Avon Park': 1, 'Polk County': 1}",{}
1,{'America': 1},{},{}
2,{},{},{}
3,{},{},{}
4,{},{},{}


In [33]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

In [34]:
ct_ner['NER-msg']

0        [Minneapolis, Polk County, Avon Park]
1                                    [America]
2                                           []
3                                           []
4                                           []
                         ...                  
23895                            [Minneapolis]
23896                                     [US]
23897           [Minneapolis, Hennepin County]
23898                                       []
23899                                       []
Name: NER-msg, Length: 23900, dtype: object

In [35]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [36]:
ct_ner['NER']

0        [Minneapolis, Polk County, Avon Park, Polk Cou...
1                                                [America]
2                                                       []
3                                                       []
4                                                       []
                               ...                        
23895              [Minneapolis, Minneapolis, Minneapolis]
23896                       [US, Minneapolis, Minneapolis]
23897    [Minneapolis, Hennepin County, Minneapolis, Mi...
23898                                                   []
23899                                        [Minneapolis]
Name: NER, Length: 23900, dtype: object

In [37]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

In [38]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [39]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER
0,0,104380826317363_2958316794257071,Facebook,2020-05-29 22:10:01,2021-06-15 16:21:42,link,Aftershocks of George Floyd's death hit Centra...,theledger.com,George Floyd attended South Florida State Coll...,George Floyd attended South Florida State Coll...,1.0,https://www.theledger.com/news/20200529/afters...,https://www.facebook.com/104380826317363/posts...,3375,-1.250000,1.0,5,7,0,0,2,0,2,0,0,0,3,2,2,2,2,3,2,4,0,1,23719,News Chief,newschief,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The News Chief is a daily newspaper in Winter ...,False,2011-04-27 15:29:39,en,0,23719|2958316794257071,"[Minneapolis, Polk County, Avon Park]","[Polk County, Avon Park]",[],"[minneapolis, avon park, polk county]"
1,1,2015249512117888_2353472281628941,Facebook,2020-05-29 22:10:00,2021-07-14 01:56:00,link,Justice — Zina for Congress,zinaforcongress.com,Protesters taking to the streets to seek justi...,"Ahmaud Arbery, Breonna Taylor and George Floyd...",2.0,http://www.zinaforcongress.com/justice,https://www.facebook.com/2015249512117888/post...,796,1.826087,1.0,31,6,0,5,0,0,0,0,0,1,6,2,2,3,2,3,2,3,0,2,8581055,Zina Spezakis for Congress NJ9,zspezakisNJ,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,Our leadership is ineffective and corrupt. It ...,False,2019-02-12 20:21:37,en,0,8581055|2353472281628941,[America],[],[],[america]
2,2,127784800585461_3342576742439568,Facebook,2020-05-29 22:10:00,2021-07-20 14:56:16,link,A Fearless Response To George Floyd’s Death | ...,fearlessfollower.org,A Fearless Response To George Floyd’s Death by...,Hey Fearless Followers! Here's a challenging r...,1.0,https://fearlessfollower.org/a-fearless-respon...,https://www.facebook.com/127784800585461/posts...,4484,3.263158,1.0,16,29,4,11,0,0,2,0,0,0,3,3,2,3,0,0,0,0,0,1,1467090,Ridge Point Community Church,ridgepointchurch,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Ridge Point exists to Follow Jesus and Fearles...,False,2010-06-14 17:44:07,en,0,1467090|3342576742439568,[],[],[],[]
3,3,7127779172_10158347173219173,Facebook,2020-05-29 22:09:59,2021-07-19 06:12:54,link,The Shooting Of Black Americans Started Long B...,pridenation.com,By Taryn Finley | HuffPost Donald Trump threat...,The Shooting Of Black Americans Started Long B...,2.0,http://ow.ly/8IiY50zUbyT,https://www.facebook.com/7127779172/posts/1015...,2459,-0.900000,1.0,0,0,0,0,0,0,1,0,0,0,1,2,1,1,1,1,2,1,0,1,8609875,PNN Media Group,pridenation,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,PHARMATIVE PNN Media Group is the parent compa...,False,2008-01-07 06:37:40,en,0,8609875|10158347173219173,[],[],[],[]
4,4,57218502143_10157744314187144,Facebook,2020-05-29 22:09:54,2021-08-06 00:24:46,link,CN2 Newscast 5/29/20 - CN2 News,cn2.com,A pastor spreading a message of unity and love...,A pastor spreading a message of unity and love...,2.0,https://www.cn2.com/cn2-newscast-5-29-20/,https://www.facebook.com/57218502143/posts/101...,24960,-6.000000,1.0,7,0,0,0,0,0,0,0,0,0,8,6,3,4,4,4,4,9,0,3,1486517,CN2 News,CN2RockHill,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,NEWS TIP? 803-326-2762 or news@cn2.com\n\n,False,2009-03-09 22:18:59,en,0,1486517|10157744314187144,[],[],[],[]
...,...,...,

In [40]:
ct_ner['NER-1']=ct_ner['NER']

In [41]:
ct_ner['NER-1'][0]

['minneapolis', 'avon park', 'polk county']

In [42]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [43]:
ct_ner[:100]

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,104380826317363_2958316794257071,Facebook,2020-05-29 22:10:01,2021-06-15 16:21:42,link,Aftershocks of George Floyd's death hit Centra...,theledger.com,George Floyd attended South Florida State Coll...,George Floyd attended South Florida State Coll...,1.0,https://www.theledger.com/news/20200529/afters...,https://www.facebook.com/104380826317363/posts...,3375,-1.250000,1.0,5,7,0,0,2,0,2,0,0,0,3,2,2,2,2,3,2,4,0,1,23719,News Chief,newschief,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The News Chief is a daily newspaper in Winter ...,False,2011-04-27 15:29:39,en,0,23719|2958316794257071,"[Minneapolis, Polk County, Avon Park]","[Polk County, Avon Park]",[],"[minneapolis, avon park, polk county]","[minneapolis, avon park, polk county]"
1,1,2015249512117888_2353472281628941,Facebook,2020-05-29 22:10:00,2021-07-14 01:56:00,link,Justice — Zina for Congress,zinaforcongress.com,Protesters taking to the streets to seek justi...,"Ahmaud Arbery, Breonna Taylor and George Floyd...",2.0,http://www.zinaforcongress.com/justice,https://www.facebook.com/2015249512117888/post...,796,1.826087,1.0,31,6,0,5,0,0,0,0,0,1,6,2,2,3,2,3,2,3,0,2,8581055,Zina Spezakis for Congress NJ9,zspezakisNJ,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,Our leadership is ineffective and corrupt. It ...,False,2019-02-12 20:21:37,en,0,8581055|2353472281628941,[America],[],[],[america],[america]
2,2,127784800585461_3342576742439568,Facebook,2020-05-29 22:10:00,2021-07-20 14:56:16,link,A Fearless Response To George Floyd’s Death | ...,fearlessfollower.org,A Fearless Response To George Floyd’s Death by...,Hey Fearless Followers! Here's a challenging r...,1.0,https://fearlessfollower.org/a-fearless-respon...,https://www.facebook.com/127784800585461/posts...,4484,3.263158,1.0,16,29,4,11,0,0,2,0,0,0,3,3,2,3,0,0,0,0,0,1,1467090,Ridge Point Community Church,ridgepointchurch,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Ridge Point exists to Follow Jesus and Fearles...,False,2010-06-14 17:44:07,en,0,1467090|3342576742439568,[],[],[],[],[]
3,3,7127779172_10158347173219173,Facebook,2020-05-29 22:09:59,2021-07-19 06:12:54,link,The Shooting Of Black Americans Started Long B...,pridenation.com,By Taryn Finley | HuffPost Donald Trump threat...,The Shooting Of Black Americans Started Long B...,2.0,http://ow.ly/8IiY50zUbyT,https://www.facebook.com/7127779172/posts/1015...,2459,-0.900000,1.0,0,0,0,0,0,0,1,0,0,0,1,2,1,1,1,1,2,1,0,1,8609875,PNN Media Group,pridenation,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,PHARMATIVE PNN Media Group is the parent compa...,False,2008-01-07 06:37:40,en,0,8609875|10158347173219173,[],[],[],[],[]
4,4,57218502143_10157744314187144,Facebook,2020-05-29 22:09:54,2021-08-06 00:24:46,link,CN2 Newscast 5/29/20 - CN2 News,cn2.com,A pastor spreading a message of unity and love...,A pastor spreading a message of unity and love...,2.0,https://www.cn2.com/cn2-newscast-5-29-20/,https://www.facebook.com/57218502143/posts/101...,24960,-6.000000,1.0,7,0,0,0,0,0,0,0,0,0,8,6,3,4,4,4,4,9,0,3,1486517,CN2 News,CN2RockHill,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,NEWS TIP? 803-326-2762 or news@cn2.com\n\n,False,2009-03-09 22

In [44]:
%cd mapping_US

/content/gdrive/My Drive/2021-online-language-offline-impact-main/mapping_US


In [45]:
!ls

 Country.txt			 dataframe0610_post.csv
 dataframe0529_post.csv		 dataframe0610_post_remove.csv
 dataframe0529_post_remove.csv	 dataframe0611_post.csv
 dataframe0530_post.csv		 dataframe0611_post_remove.csv
 dataframe0530_post_remove.csv	 dataframe_mapping
 dataframe0531_post.csv		 dataframe_mapping_0529.csv
 dataframe0531_post_remove.csv	 dataframe_mapping_0530.csv
 dataframe0601_post.csv		 dataframe_mapping_0531.csv
 dataframe0601_post_remove.csv	 dataframe_mapping_0601.csv
 dataframe0602_post.csv		 dataframe_mapping_0602.csv
 dataframe0602_post_remove.csv	 dataframe_mapping_0603.csv
 dataframe0603_post.csv		 dataframe_mapping_0604.csv
 dataframe0603_post_remove.csv	 dataframe_mapping_0605.csv
 dataframe0604_post.csv		 dataframe_mapping_0606.csv
 dataframe0604_post_remove.csv	 dataframe_mapping_0607.csv
 dataframe0605_post.csv		 dataframe_mapping_0608.csv
 dataframe0605_post_remove.csv	 dataframe_mapping_0609.csv
 dataframe0606_post.csv		 dataframe_mapping_0610.csv
 dataframe0606_post

In [46]:
ct_mapping=pd.read_csv('dataframe_mapping_0529.csv',encoding='ISO-8859-1')

In [47]:
ct_mapping

,Unnamed: 0,City,Number,Mapping,Latitude,Longtitude
0,0,minneapolis,16071,"Minneapolis, Hennepin County, Minnesota, 55488...",44.977300,-93.265469
1,1,minnesota,4009,"Minnesota, United States",45.989659,-94.611329
2,2,america,1044,"America, Horst aan de Maas, Limburg, Nederland",51.437148,5.979900
3,3,st. paul,919,"Saint Paul, Ramsey County, Minnesota, United S...",44.949749,-93.093103
4,4,hennepin county,761,"Hennepin County, Minnesota, United States",45.025723,-93.486505
...,...,...,...,...,...,...
1554,1554,river | savannah,1,"Savannah, Pearl River County, Mississippi, Uni...",30.739633,-89.490895
1555,1555,the city failed us',1,NaN,NaN,NaN
1556,1556,new yorker)background,1,NaN,NaN,NaN
1557,1557,black community,1,"Black, Berastegui kalea, Abando, Bilbao, Bizka...",43.262258,-2.927797


In [48]:
country_list=pd.read_csv('Country.txt')
country_list['Name']=country_list['Name'].str.lower()

In [49]:
country_list['Name']

0            afghanistan
1          �land islands
2                albania
3                algeria
4         american samoa
             ...        
250                   us
251    the united states
252              america
253                  u.s
254        united states
Name: Name, Length: 255, dtype: object

In [50]:
states_list=pd.read_csv('states.csv')
states_list['State']

0                             Alabama, United States
1                              Alaska, United States
2                             Arizona, United States
3                            Arkansas, United States
4                          California, United States
                           ...                      
165         Armagh, Northern Ireland, United Kingdom
166           Down, Northern Ireland, United Kingdom
167      Fermanagh, Northern Ireland, United Kingdom
168    Londonderry, Northern Ireland, United Kingdom
169         Tyrone, Northern Ireland, United Kingdom
Name: State, Length: 170, dtype: object

In [51]:
from numpy.core.numeric import NaN
for i in range(len(ct_mapping['City'])):
  for k in range(len(country_list['Name'])):
    if ct_mapping['City'][i]==country_list['Name'][k]:
      ct_mapping['Mapping'][i]=NaN

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [52]:
for i in range(len(ct_mapping['City'])):
  for k in range(len(states_list['State'])):
    if ct_mapping['Mapping'][i]==states_list['State'][k]:
      ct_mapping['Mapping'][i]=NaN

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [53]:
for i in range(len(ct_mapping['City'])):
  for k in range(len(country_list['Name'])):
    if ct_mapping['Mapping'][i]==country_list['Name'][k]:
      ct_mapping['Mapping'][i]=NaN

In [54]:
pd.set_option('display.max_rows', None)

In [55]:
ct_mapping

,Unnamed: 0,City,Number,Mapping,Latitude,Longtitude
0,0,minneapolis,16071,"Minneapolis, Hennepin County, Minnesota, 55488...",44.977300,-93.265469
1,1,minnesota,4009,NaN,45.989659,-94.611329
2,2,america,1044,NaN,51.437148,5.979900
3,3,st. paul,919,"Saint Paul, Ramsey County, Minnesota, United S...",44.949749,-93.093103
4,4,hennepin county,761,"Hennepin County, Minnesota, United States",45.025723,-93.486505
5,5,the united states,531,NaN,29.633008,-97.988594
6,6,denver,515,"City and County of Denver, Colorado, United St...",39.739236,-104.984862
7,7,u.s.,492,NaN,48.630579,9.417545
8,8,us,463,NaN,39.783730,-100.445882
9,9,houston,379,"Houston, Harris County, Texas, 77002-5902, Uni...",29.758938,-95.367697


In [56]:
ct_mapping

,Unnamed: 0,City,Number,Mapping,Latitude,Longtitude
0,0,minneapolis,16071,"Minneapolis, Hennepin County, Minnesota, 55488...",44.977300,-93.265469
1,1,minnesota,4009,NaN,45.989659,-94.611329
2,2,america,1044,NaN,51.437148,5.979900
3,3,st. paul,919,"Saint Paul, Ramsey County, Minnesota, United S...",44.949749,-93.093103
4,4,hennepin county,761,"Hennepin County, Minnesota, United States",45.025723,-93.486505
5,5,the united states,531,NaN,29.633008,-97.988594
6,6,denver,515,"City and County of Denver, Colorado, United St...",39.739236,-104.984862
7,7,u.s.,492,NaN,48.630579,9.417545
8,8,us,463,NaN,39.783730,-100.445882
9,9,houston,379,"Houston, Harris County, Texas, 77002-5902, Uni...",29.758938,-95.367697


In [57]:
ct_mapping.to_csv('dataframe_mapping_0529.csv')

In [58]:
#ct_mapping=pd.read_csv('dataframe_mapping_0530.csv',encoding='ISO-8859-1')

In [59]:
for k in range(len(ct_ner['NER-1'])):
  list=[]
  for i in ct_ner['NER-1'][k]:
    for j in range(len(ct_mapping['City'])):
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  ct_ner['NER-1'][k]=list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [60]:
ct_ner[:100]

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,104380826317363_2958316794257071,Facebook,2020-05-29 22:10:01,2021-06-15 16:21:42,link,Aftershocks of George Floyd's death hit Centra...,theledger.com,George Floyd attended South Florida State Coll...,George Floyd attended South Florida State Coll...,1.0,https://www.theledger.com/news/20200529/afters...,https://www.facebook.com/104380826317363/posts...,3375,-1.250000,1.0,5,7,0,0,2,0,2,0,0,0,3,2,2,2,2,3,2,4,0,1,23719,News Chief,newschief,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The News Chief is a daily newspaper in Winter ...,False,2011-04-27 15:29:39,en,0,23719|2958316794257071,"[Minneapolis, Polk County, Avon Park]","[Polk County, Avon Park]",[],"[minneapolis, avon park, polk county]","[Minneapolis, Hennepin County, Minnesota, 5548..."
1,1,2015249512117888_2353472281628941,Facebook,2020-05-29 22:10:00,2021-07-14 01:56:00,link,Justice — Zina for Congress,zinaforcongress.com,Protesters taking to the streets to seek justi...,"Ahmaud Arbery, Breonna Taylor and George Floyd...",2.0,http://www.zinaforcongress.com/justice,https://www.facebook.com/2015249512117888/post...,796,1.826087,1.0,31,6,0,5,0,0,0,0,0,1,6,2,2,3,2,3,2,3,0,2,8581055,Zina Spezakis for Congress NJ9,zspezakisNJ,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,Our leadership is ineffective and corrupt. It ...,False,2019-02-12 20:21:37,en,0,8581055|2353472281628941,[America],[],[],[america],[nan]
2,2,127784800585461_3342576742439568,Facebook,2020-05-29 22:10:00,2021-07-20 14:56:16,link,A Fearless Response To George Floyd’s Death | ...,fearlessfollower.org,A Fearless Response To George Floyd’s Death by...,Hey Fearless Followers! Here's a challenging r...,1.0,https://fearlessfollower.org/a-fearless-respon...,https://www.facebook.com/127784800585461/posts...,4484,3.263158,1.0,16,29,4,11,0,0,2,0,0,0,3,3,2,3,0,0,0,0,0,1,1467090,Ridge Point Community Church,ridgepointchurch,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Ridge Point exists to Follow Jesus and Fearles...,False,2010-06-14 17:44:07,en,0,1467090|3342576742439568,[],[],[],[],[]
3,3,7127779172_10158347173219173,Facebook,2020-05-29 22:09:59,2021-07-19 06:12:54,link,The Shooting Of Black Americans Started Long B...,pridenation.com,By Taryn Finley | HuffPost Donald Trump threat...,The Shooting Of Black Americans Started Long B...,2.0,http://ow.ly/8IiY50zUbyT,https://www.facebook.com/7127779172/posts/1015...,2459,-0.900000,1.0,0,0,0,0,0,0,1,0,0,0,1,2,1,1,1,1,2,1,0,1,8609875,PNN Media Group,pridenation,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,PHARMATIVE PNN Media Group is the parent compa...,False,2008-01-07 06:37:40,en,0,8609875|10158347173219173,[],[],[],[],[]
4,4,57218502143_10157744314187144,Facebook,2020-05-29 22:09:54,2021-08-06 00:24:46,link,CN2 Newscast 5/29/20 - CN2 News,cn2.com,A pastor spreading a message of unity and love...,A pastor spreading a message of unity and love...,2.0,https://www.cn2.com/cn2-newscast-5-29-20/,https://www.facebook.com/57218502143/posts/101...,24960,-6.000000,1.0,7,0,0,0,0,0,0,0,0,0,8,6,3,4,4,4,4,9,0,3,1486517,CN2 News,CN2RockHill,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,NEWS TIP? 803-326-2762 or news@cn2.com\n\n,False,2009-

In [61]:
ct_ner['NER-1'][:100]

0     [Minneapolis, Hennepin County, Minnesota, 5548...
1                                                 [nan]
2                                                    []
3                                                    []
4                                                    []
5                                                    []
6     [Minneapolis, Hennepin County, Minnesota, 5548...
7                                                    []
8     [Minneapolis, Hennepin County, Minnesota, 5548...
9     [Minneapolis, Hennepin County, Minnesota, 5548...
10    [Minneapolis, Hennepin County, Minnesota, 5548...
11    [Minneapolis, Hennepin County, Minnesota, 5548...
12    [Minneapolis, Hennepin County, Minnesota, 5548...
13    [Fontana, San Bernardino County, California, 9...
14    [Minneapolis, Hennepin County, Minnesota, 5548...
15    [Oklahoma County, Oklahoma, United States, Okl...
16                                                   []
17    [Minneapolis, Hennepin County, Minnesota, 

In [62]:
ct_ner.to_csv('dataframe0529_post_remove.csv')

In [63]:
!ls

 Country.txt			 dataframe0610_post.csv
 dataframe0529_post.csv		 dataframe0610_post_remove.csv
 dataframe0529_post_remove.csv	 dataframe0611_post.csv
 dataframe0530_post.csv		 dataframe0611_post_remove.csv
 dataframe0530_post_remove.csv	 dataframe_mapping
 dataframe0531_post.csv		 dataframe_mapping_0529.csv
 dataframe0531_post_remove.csv	 dataframe_mapping_0530.csv
 dataframe0601_post.csv		 dataframe_mapping_0531.csv
 dataframe0601_post_remove.csv	 dataframe_mapping_0601.csv
 dataframe0602_post.csv		 dataframe_mapping_0602.csv
 dataframe0602_post_remove.csv	 dataframe_mapping_0603.csv
 dataframe0603_post.csv		 dataframe_mapping_0604.csv
 dataframe0603_post_remove.csv	 dataframe_mapping_0605.csv
 dataframe0604_post.csv		 dataframe_mapping_0606.csv
 dataframe0604_post_remove.csv	 dataframe_mapping_0607.csv
 dataframe0605_post.csv		 dataframe_mapping_0608.csv
 dataframe0605_post_remove.csv	 dataframe_mapping_0609.csv
 dataframe0606_post.csv		 dataframe_mapping_0610.csv
 dataframe0606_post